In [32]:
lister=['street','circle','cir','drive',
 'dr','road','rd','yes',
 'no','no','yes','hey',
 'street','street','streeet','street']

In [47]:
def swaptoken(tokenlist,toswap,swapwith):
    returnlist=[]
    for i in tokenlist:
        if i==toswap:
            returnlist.append(swapwith)
        else:
            returnlist.append(i)
    return (returnlist)

In [48]:
swaptoken(lister,'street','st')

['st',
 'circle',
 'cir',
 'drive',
 'dr',
 'road',
 'rd',
 'yes',
 'no',
 'no',
 'yes',
 'hey',
 'st',
 'st',
 'streeet',
 'st']

In [6]:
import pyodbc 
import pandas as pd
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=VisionDB;'
                      'Database=Sandbox_Vision;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

In [7]:
query = """ SELECT VE.Name
      ,VEA.[Vendor]
      ,VEA.[Address]
      ,VEA.[Address1]
      ,VEA.[Address2]
      ,VEA.[Address3]
      ,VEA.[Address4]
      ,VEA.[City]
      ,VEA.[State]
      ,VEA.[ZIP]
  FROM [Sandbox_Vision].[dbo].[VEAddress] VEA
  inner join [Sandbox_Vision].[dbo].VE 
  ON VEA.Vendor=VE.Vendor
  WHERE VE.LinkedClient IS NULL
  """

In [8]:
Vendors = pd.read_sql_query(query,conn)

In [11]:
Vendors['CleanedAddress']=(Vendors['Name']+" "+Vendors['Address1']+" "+Vendors['City']+" "+Vendors['State']+" "+Vendors['ZIP']).\
    str.lower()

In [2]:
import nltk

In [9]:
def get_ngrams(series: pd.Series,n:int):
    ngrams = series.copy().str.split(' ').explode()
    ngrams1 = series.copy().str.split(' ').explode()
    for i in range(1,n):
        foo=ngrams1.groupby(level=0).shift(-i)
        ngrams=ngrams.str.cat(foo,sep=' ')
    ngrams = ngrams.dropna()
    return pd.DataFrame(ngrams)

def get_multigrams(series,n):
    grams=get_ngrams(series,1)
    if n==1:
        return grams
    else:
        for i in range(1,n+1):
            if i==1:
                continue
            grams=grams.append(get_ngrams(series,i))
    return grams

In [5]:
def tokennamefilter(tokenlist,minlen=2):
    tokens = [x for x in tokenlist if x not in ['mod','bradley','corp','pdx','generic','tpb','modd','kohler','sloan','bobrick','lucerne','dlr','dyson']]
    return (tokens)

In [ ]:
#po->p.o.
#drive -> dr
#street -> st
#avenue -> ave
#boulevard->blvd
#road -> rd
#north -> n
#south -> s
#west -> w
#east -> e
#highway -> hwy
#llc -> ''

In [49]:
get_ngrams(Vendors.drop_duplicates(subset=['Vendor'])['Address1'].str.lower().str.replace('.','').str.replace(',',''),1).\
           assign(n=1).groupby(['Address1'])['n'].sum().\
           reset_index().query('n>30').sort_values(['n'],ascending=False).head(60)

C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


,Address1,n
11067,street,2101
6568,box,1800
10196,po,1376
11096,suite,1121
6291,avenue,1018
10555,road,715
7549,drive,647
6284,ave,578
10946,south,537
11671,west,453


In [20]:
Vendors.drop_duplicates(subset=['Vendor'])

,Name,Vendor,Address,Address1,Address2,Address3,Address4,City,State,ZIP,CleanedAddress
0,"Jimmy John's, LLC",V12366,72nd Street,300 South 72 Street,None,None,None,Omaha,NE,68106,"jimmy john's, llc 300 south 72 street omaha ne..."
1,Minnesota Department of Revenue,11867,2010 Audit Address,Attn: Christopher Determan,PO Box 21367,None,None,Eagan,MN,55121-0367,minnesota department of revenue attn: christop...
2,Department of Homeland Security,WDEP05,CA,USCIS California Service Center,"24000 Avila Road, 2nd Floor Room 2312",None,None,Laquna Niguel,CA,92677,department of homeland security uscis californ...
3,Mike Beard Volunteer Committee,11975,<Default>,"% AIA, Minnesota","275 Market Street, Suite 54",None,None,Minneapolis,MN,55405,"mike beard volunteer committee % aia, minnesot..."
4,AT&T,1005,<Default>,Bill Payment Center,None,None,None,Chicago,IL,60663-0001,at&t bill payment center chicago il 60663-0001
...,...,...,...,...,...,...,...,...,...,...,...
16661,Umaya Lighting Design DMCC,V19893,<Default>,P.O. Box 30243,None,None,None,None,Dubai,None,NaN
16662,First National Bank Kenneth P. West Custody IR...,V19905,<Default>,"14010 FNB Parkway, Suite 200",None,None,None,Omaha,NE,68154,first national bank kenneth p. west custody ir...
16663,"Great Western Bank, FBO Thomas E. Penney",V19907,<Default>,IRA Account #7803,"9290 West Dodge Road, Suite 202",None,None,Omaha,NE,68114,"great western bank, fbo thomas e. penney ira a..."
16664,Committee to Improve Rescue Union Schools-Yes ...,V19899,2980,2980 Fairchild Dr,None,None,None,El Dorado Hills,CA,95762,committee to improve rescue union schools-yes ...
